In [41]:
#%pip install lightkurve
#%pip install pyarrow

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk
import pandas as pd
import requests
import os
import time
%matplotlib qt


In [43]:
def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        
        result = func(*args, **kwargs)  # Call the original function
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"{func.__name__} took {elapsed_time} seconds to run.")
        
        return result  # Return the result of the original function
    return wrapper

In [44]:
@timeit
def cut_time_series(lc_time: np.array, lc_flux: np.array, starting_transit:int, number_of_transits:int, T0: float, period: float):
    
    interval = 0.5 # day
    values_either_side_of_T0 = (lc_time<T0+interval) & (lc_time>T0-interval)
    if values_either_side_of_T0.sum() == 0:
        new_T0 = find_starting_transit(lc_time,T0, period)
        if new_T0==T0:
            print('Cannot continue, cannot cut')
        else:
            T0 = new_T0
    
    number_of_transits = number_of_transits-starting_transit

    transit_start=(T0+(period*starting_transit))
    transit_end=(T0+(period*int(number_of_transits)))
    
    time_cut_mask = (lc_time>=transit_start) & (lc_time<=transit_end)
    lc_time_cut = lc_time[time_cut_mask]
    lc_flux_cut = lc_flux[time_cut_mask]
    return lc_time_cut, lc_flux_cut

In [45]:
@timeit
def phase_fold(lc_time: np.array, period:float ):
    timestep = lc_time[2] - lc_time[1] # just in case the first value is NaN
    ts_length = len(lc_time)
    
    phase=np.mod((lc_time-lc_time.min())+period/2, period)/period
    phis = 2.*np.pi*np.mod((lc_time-lc_time.min())/(period),1) 
    return phis,phase

In [46]:
# import pandas as pd
# kepler_list = pd.read_csv('kep_conf_names_2023_02_18_15_00_44.csv', delimiter=',', usecols = ['pl_name'])
# K2_list = pd.read_csv('k2.csv', delimiter=',', usecols = ['pl_name','period'])
# Tess_list = pd.read_csv('TESS_2023.csv', delimiter=',', usecols = ['pl_name','hostname'])

# planet_list = pd.concat([kepler_list,Tess_list,K2_list], axis=0)

In [47]:
@timeit
def download_lightcurves(planet_name, author):
    search_results = lk.search_lightcurve(planet_name, author=author, cadence="long")
    
    if len(search_results)==0:
        print(f'No Data for {planet_name}')
        return [], []
    lc_time_list = []
    lc_flux_list = []
    for lc in search_results:
        light_curve = lc.download()
        
        lc_mean = np.mean(light_curve.flux.value)
        quarter_normalised_flux = light_curve.flux.value.astype(np.ndarray) / np.mean(light_curve.flux.value)
        quarter_lc_time = light_curve.time.mjd
        lc_time_list.append(quarter_lc_time.astype(float))
        lc_flux_list.append(quarter_normalised_flux.astype(float))
    lc_time = np.concatenate(lc_time_list).astype(float)
    lc_flux = np.concatenate(lc_flux_list).astype(float)
    
    lc_flux[np.isnan(lc_flux)]=1
    
    lc_flux -= np.mean(lc_flux)
    
    return lc_time,lc_flux


In [48]:
def plot_fitted_data(phase,fit,old_data,mean_outside_trojan_region,std_outside_trojan_region,std_devs,outer_l4,inner_l4,inner_l5,outer_l5):
    plt.figure()
    plt.plot(phase,fit,'r')
    plt.axhline(mean_outside_trojan_region)
    plt.axhline(mean_outside_trojan_region-std_outside_trojan_region*std_devs)
    plt.axvline(outer_l4)
    plt.axvline(inner_l4)
    plt.axvline(inner_l5)
    plt.axvline(outer_l5)
    plt.plot(phase,old_data,',b')
    #plt.ylim(min_outside_trojan_region,max_outside_trojan_region)
    return

@timeit
def get_trojan_regions(phase, fit ,outer_l4,inner_l4, inner_l5, outer_l5):
    in_trojon_region_l5 = (phase>inner_l5) & (phase<outer_l5)
    in_trojon_region_l4 = (phase>outer_l4) & (phase<inner_l4)

    outside_trojan_and_planet = (phase<outer_l4) | (phase>outer_l5)

    in_trojon_region = in_trojon_region_l4 | in_trojon_region_l5

    mean_outside_trojan_region = np.mean(fit[outside_trojan_and_planet])
    max_outside_trojan_region = np.max(fit[outside_trojan_and_planet|in_trojon_region_l4|in_trojon_region_l5])
    min_outside_trojan_region = np.min(fit[outside_trojan_and_planet|in_trojon_region_l4|in_trojon_region_l5])
    std_outside_trojan_region = np.std(fit[outside_trojan_and_planet])
    return mean_outside_trojan_region, max_outside_trojan_region, min_outside_trojan_region,std_outside_trojan_region,outside_trojan_and_planet,in_trojon_region

def format_df(df):
    df = df.T
    df.columns = ['Time', 'Flux']
    df = df.set_index('Time')
    return df

In [49]:
#%pip install tensorflow 
import tensorflow.compat.v1 as tf
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
#   try:
#     tf.config.set_visible_devices(gpus[0], 'GPU')
#     tf.config.experimental.set_memory_growth(gpus[0], True)
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#   except RuntimeError as e:
#     # Visible devices must be set before GPUs have been initialized
#     print(e)

In [50]:
def get_kepler_planet_data(kepler_name):
    url = f"https://archive.stsci.edu/kepler/confirmed_planets/search.php?kepler_name={kepler_name.replace(' ','+')}&max_records=1&action=Search&outputformat=JSON&coordformat=dec&verb=3"
    r = requests.get(url).json()[0]
    period = float(r['Period'])
    T0 = float(r['Time of transit'])
    return period, T0

def get_k2_planet_data(k2_name):
    url = f"https://archive.stsci.edu/k2/published_planets/search.php?k2_name={k2_name.replace(' ','+')}&max_records=1&action=Search&outputformat=JSON&coordformat=dec&verb=3"
    r = requests.get(url).json()[0]
    period = float(r['Period'])
    T0 = float(r['Time of transit'])
    return period, T0

In [51]:
@timeit
def harmonics_tf(cut_time, cut_flux, cut_phis, cut_phase, period, number_of_transits):
    """
    TensorFlow optimized version
    """
    ts_length = len(cut_time)
    
    nharms = tf.linspace(1.0, ts_length/number_of_transits, ts_length//number_of_transits)
    
    cut_flux_tensor = tf.reshape(tf.convert_to_tensor(cut_flux, dtype=tf.float32), [ts_length, 1])
    
    pmat = nharms * tf.reshape(tf.convert_to_tensor(cut_phis, dtype=tf.float32), [ts_length, 1])

    C = tf.math.cos(pmat)
    S = tf.math.sin(pmat)

    Tmat = tf.concat([C, S], axis=1)
    R = tf.linalg.inv(tf.matmul(tf.transpose(Tmat), Tmat))
    R = tf.matmul(R, tf.matmul(tf.transpose(Tmat), cut_flux_tensor))
    
    fit = tf.reshape(tf.matmul(Tmat, R), [ts_length])
    
    isort = tf.argsort(cut_phase)
    phase = tf.gather(cut_phase, isort)
    fit = tf.gather(fit, isort)
    cut_flux_ordered = tf.gather(cut_flux, isort)
    
    return fit, cut_flux_ordered, phase

In [52]:
@timeit
def harmonics_np(cut_time, cut_flux, cut_phis, cut_phase, period, number_of_transits):
    """
    Do not touch this, it works!
    """
    ts_length = len(cut_time)
    
    nharms = np.linspace(1,ts_length/number_of_transits,ts_length//number_of_transits)
    
    cut_flux_mat = np.matrix(np.array(cut_flux).reshape(ts_length,1)) # DO NOT CHANGE THIS!! I SPENT HOURS TRYING TO RESOLVE THE ASTROPY NDTYPE
    
    pmat = nharms*cut_phis.reshape(ts_length, 1)

    C = np.cos(pmat)
    S = np.sin(pmat)

    Tmat = np.matrix(np.hstack((C,S))) # computationally intense calc
    R = (Tmat.T*Tmat).I * (Tmat.T*cut_flux_mat)

    fit = np.array(Tmat*R).reshape(ts_length)

    # Now plot phase-folded
    isort = np.argsort(cut_phase)
    phase = cut_phase[isort]
    fit = fit[isort]
    cut_flux_ordered = cut_flux[isort]
    return fit,cut_flux_ordered, phase


In [53]:
@timeit
def find_starting_transit(lc_time,T0, period):
    """ the T0 I have may not be exactly at the start, or even in the time series
    So I need to find what an appropriate starting point sould be
    """
    new_starting_T0 = T0
    if (lc_time<T0).sum()>0: 
        
        for transit in np.arange(100,1,-1):
            if (lc_time<(T0-(period*transit))).sum()>0:
                new_starting_T0 = T0-(period*transit)
                break
        
    else:
        for transit in np.arange(1,100,1):
            if (lc_time<(T0+(period*transit))).sum()>0:
                new_starting_T0 = T0+(period*transit)
                break
    return new_starting_T0
        
    

In [54]:
def is_any_value_nan(numeric_list):
    """
    Cheks if any value in a list is nan. 
    """
    for val in numeric_list:
        if val == np.nan or val == 'nan' or np.isnan(val)==True:
            return True
    return False



In [55]:
import pandas as pd
df_planets = pd.read_csv('planets_filtered_with_T0.csv')

In [56]:
df_planets = df_planets.dropna()

In [57]:
std_devs = 3
starting_transit = 0
outer_l4,inner_l4, inner_l5, outer_l5 = 0.24, 0.42, 0.57, 0.75
assumed_planet_duration = 4 #days

In [58]:
import time 
mask =  (df_planets['Author'].str.contains('K2')) & (df_planets['period']>50)
for ind, row in df_planets[mask].head(10).iterrows():
    print('##################')
    #i = 2824
    #row = df_planets.iloc[i]
    planet_name = row['pl_name']

    period = row['period']
    T0 = row['T0']
    author = row['Author']

    print(planet_name, "period:", period, "T0:", T0)
    if is_any_value_nan([float(period), float(T0)])==True:
        print("Nan found")
        continue
    
    lc_time, lc_flux = download_lightcurves(planet_name, author)

    if len(lc_time)==0:
        print('No LC Data')
        continue

    number_of_transits = int((np.max(lc_time) - np.min(lc_time))/period)-1
    number_of_transits = 300 if number_of_transits>300 else number_of_transits

    lc_time_cut, lc_flux_cut = cut_time_series(lc_time,
                                            lc_flux,
                                            starting_transit,
                                            number_of_transits,
                                            T0,
                                            period)
    phis,phase = phase_fold(lc_time_cut, period ) ##plt.plot(lc_time_cut, lc_flux_cut, ',b') #plt.plot(phase, lc_flux_cut, ',b')

    ts_length = len(lc_time_cut)

    #     lc_time_cut = tf.constant(lc_time_cut, np.float32) # uncomment if you want to use tensorflow
    #     lc_flux_cut = tf.constant(lc_flux_cut, np.float32)
    #     phis = tf.constant(phis, np.float32)
    #     phase = tf.constant(phase, np.float32)

    t1 = time.time()
    fit, old_data_ordered, phase = harmonics_tf(lc_time_cut,
                                                    lc_flux_cut,
                                                    phis,
                                                    phase,
                                                    period,
                                                    number_of_transits)
    print(time.time()-t1)
    mean_outside_trojan_region, max_outside_trojan_region, min_outside_trojan_region,std_outside_trojan_region,outside_trojan_and_planet,in_trojon_region = get_trojan_regions(phase, fit, outer_l4,inner_l4, inner_l5, outer_l5)

    #plot_fitted_data(phase,fit,old_data_ordered,mean_outside_trojan_region,std_outside_trojan_region,std_devs,outer_l4,inner_l4,inner_l5,outer_l5)

    if np.sum(fit[in_trojon_region]<mean_outside_trojan_region-std_outside_trojan_region*std_devs)>0:
        print('Something detected')

        df_save = pd.DataFrame([lc_time_cut,old_data_ordered,fit, phase]).astype(np.float32)
        df_save = df_save.T
        df_save.columns = ['Time', 'Original_norm_flux','fit', 'phase']
        df_save.to_parquet(f'{planet_name}_{period}_{T0}_fit.parquet', index=False)
        print('Saved')
    else:
        print('Nothing detected')

##################
K2-118 b period: 50.92092 T0: 57159.9701
download_lightcurves took 0.466156005859375 seconds to run.
cut_time_series took 0.0 seconds to run.
phase_fold took 0.0 seconds to run.
harmonics_tf took 0.16146445274353027 seconds to run.
0.16146445274353027


get_trojan_regions took 0.04863858222961426 seconds to run.
Nothing detected
##################
K2-185 c period: 52.713494 T0: 57158.6568
download_lightcurves took 8.482665538787842 seconds to run.
cut_time_series took 0.0 seconds to run.
phase_fold took 0.00551605224609375 seconds to run.
harmonics_tf took 0.16085171699523926 seconds to run.
0.16085171699523926
get_trojan_regions took 0.017826318740844727 seconds to run.
Nothing detected
##################
K2-263 b period: 50.8213 T0: 58110.6274
download_lightcurves took 8.444388151168823 seconds to run.
cut_time_series took 0.0 seconds to run.
phase_fold took 0.0015134811401367188 seconds to run.
harmonics_tf took 0.07628774642944336 seconds to run.
0.07628774642944336
get_trojan_regions took 0.008228540420532227 seconds to run.
Something detected
Saved


In [59]:
parquet_files = os.listdir('.')
file_names = [f for f in parquet_files if f.endswith('.parquet') ]
for file_name in file_names:
    print(file_name)
    df5 = pd.read_parquet(file_name)
    mean_outside_trojan_region, max_outside_trojan_region, min_outside_trojan_region,std_outside_trojan_region,outside_trojan_and_planet,in_trojon_region = get_trojan_regions(df5.phase, df5.fit ,outer_l4,inner_l4, inner_l5, outer_l5)
    plot_fitted_data(df5.phase,df5.fit,df5.Original_norm_flux,mean_outside_trojan_region,std_outside_trojan_region,std_devs,outer_l4,inner_l4,inner_l5,outer_l5)


K2-10 b_19.30481_56819.0775_fit.parquet
get_trojan_regions took 0.0038657188415527344 seconds to run.
K2-104 b_1.974238_57139.87998_fit.parquet
get_trojan_regions took 0.0 seconds to run.


K2-107 b_3.31392_57304.02451_fit.parquet
get_trojan_regions took 0.0 seconds to run.
K2-11 b_39.8478_56829.3511_fit.parquet
get_trojan_regions took 0.0 seconds to run.
K2-263 b_50.8213_58110.6274_fit.parquet
get_trojan_regions took 0.0064105987548828125 seconds to run.
Kepler-1000 b_120.0181272_54966.689_fit.parquet
get_trojan_regions took 0.005522727966308594 seconds to run.
Kepler-1016 c_105.6551369_55062.6_fit.parquet
get_trojan_regions took 0.0 seconds to run.
Kepler-103 c_179.612_55127.82581_fit.parquet
get_trojan_regions took 0.0 seconds to run.
Kepler-1036 b_122.8808058_54997.957_fit.parquet
get_trojan_regions took 0.015647411346435547 seconds to run.
